In [ ]:
!pip install -qq medmnist

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.3/88.3 kB 7.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


## Imports

In [ ]:
import os
import io
import imageio
import medmnist
import ipywidgets
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# from tensorflow.python.framework.ops import disable_eager_execution

# disable_eager_execution()



# Setting seed for reproducibility
SEED = 42
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
keras.utils.set_random_seed(SEED)

In [ ]:
# DATA
DATASET_NAME = "organmnist3d"
BATCH_SIZE = 32
AUTO = tf.data.AUTOTUNE
INPUT_SHAPE = (28, 28, 28, 1)
NUM_CLASSES = 11

# OPTIMIZER
LEARNING_RATE = 1e-4
WEIGHT_DECAY = 1e-5

# TRAINING
EPOCHS = 60

# TUBELET EMBEDDING
PATCH_SIZE = (4, 4, 4)
NUM_PATCHES = (INPUT_SHAPE[0] // PATCH_SIZE[0]) ** 2

# ViViT ARCHITECTURE
LAYER_NORM_EPS = 1e-6
PROJECTION_DIM = 128
NUM_HEADS = 8
NUM_LAYERS = 2

In [ ]:

def download_and_prepare_dataset(data_info: dict):
    """Utility function to download the dataset.

    Arguments:
        data_info (dict): Dataset metadata.
    """
    data_path = keras.utils.get_file(origin=data_info["url"], md5_hash=data_info["MD5"])

    with np.load(data_path) as data:
        # Get videos
        train_videos = data["train_images"]
        valid_videos = data["val_images"]
        test_videos = data["test_images"]

        # Get labels
        train_labels = data["train_labels"].flatten()
        valid_labels = data["val_labels"].flatten()
        test_labels = data["test_labels"].flatten()

    return (
        (train_videos, train_labels),
        (valid_videos, valid_labels),
        (test_videos, test_labels),
    )


# Get the metadata of the dataset
info = medmnist.INFO[DATASET_NAME]

# Get the dataset
prepared_dataset = download_and_prepare_dataset(info)
(train_videos, train_labels) = prepared_dataset[0]
(valid_videos, valid_labels) = prepared_dataset[1]
(test_videos, test_labels) = prepared_dataset[2]

32657407/32657407 [==============================] - 102s 3us/step


### `tf.data` pipeline

In [ ]:

@tf.function
def preprocess(frames: tf.Tensor, label: tf.Tensor):
    """Preprocess the frames tensors and parse the labels."""
    # Preprocess images
    frames = tf.image.convert_image_dtype(
        frames[
            ..., tf.newaxis
        ],  # The new axis is to help for further processing with Conv3D layers
        tf.float32,
    )
    # Parse label
    label = tf.cast(label, tf.float32)
    return frames, label


def prepare_dataloader(
    videos: np.ndarray,
    labels: np.ndarray,
    loader_type: str = "train",
    batch_size: int = BATCH_SIZE,
):
    """Utility function to prepare the dataloader."""
    dataset = tf.data.Dataset.from_tensor_slices((videos, labels))

    if loader_type == "train":
        dataset = dataset.shuffle(BATCH_SIZE * 2)

    dataloader = (
        dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
        .batch(batch_size)
        .prefetch(tf.data.AUTOTUNE)
    )
    return dataloader


trainloader = prepare_dataloader(train_videos, train_labels, "train")
validloader = prepare_dataloader(valid_videos, valid_labels, "valid")
testloader = prepare_dataloader(test_videos, test_labels, "test")

In [ ]:

class TubeletEmbedding(layers.Layer):
    def __init__(self, embed_dim, patch_size, **kwargs):
        super().__init__(**kwargs)
        self.projection = layers.Conv3D(
            filters=embed_dim,
            kernel_size=patch_size,
            strides=patch_size,
            padding="VALID",
        )
        self.flatten = layers.Reshape(target_shape=(-1, embed_dim))

    def call(self, videos):
        projected_patches = self.projection(videos)
        flattened_patches = self.flatten(projected_patches)
        return flattened_patches


In [ ]:

class PositionalEncoder(layers.Layer):
    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        _, num_tokens, _ = input_shape
        self.position_embedding = layers.Embedding(
            input_dim=num_tokens, output_dim=self.embed_dim
        )
        self.positions = tf.range(start=0, limit=num_tokens, delta=1)

    def call(self, encoded_tokens):
        # Encode the positions and add it to the encoded tokens
        encoded_positions = self.position_embedding(self.positions)
        encoded_tokens = encoded_tokens + encoded_positions
        return encoded_tokens


In [ ]:
#model 2
def create_vivit_classifier(
    tubelet_embedder,
    positional_encoder,
    input_shape=INPUT_SHAPE,
    transformer_layers=NUM_LAYERS,
    num_heads=NUM_HEADS,
    embed_dim=PROJECTION_DIM,
    layer_norm_eps=LAYER_NORM_EPS,
    num_classes=NUM_CLASSES,
):
    # Get the input layer
    inputs = layers.Input(shape=input_shape)
    # print(inputs.shape)
    # Create patches.
    patches = tubelet_embedder(inputs)
    # print(patches.shape)
    # Encode patches.
    encoded_patches = positional_encoder(patches)
    # print(encoded_patches.shape)


    # Create multiple layers of the Transformer block.
    representation_s = [None]*(INPUT_SHAPE[0] // PATCH_SIZE[0]) #number of time transformers needed

    for i in range(INPUT_SHAPE[0] // PATCH_SIZE[0]):
      temp = encoded_patches[:,i*(INPUT_SHAPE[1] // PATCH_SIZE[1])*(INPUT_SHAPE[2] // PATCH_SIZE[2]):(i+1)*(INPUT_SHAPE[1] // PATCH_SIZE[1])*(INPUT_SHAPE[2] // PATCH_SIZE[2]), :]
      for _ in range(transformer_layers):
          # Layer normalization and MHSA
          x1 = layers.LayerNormalization(epsilon=1e-6)(temp)
          attention_output = layers.MultiHeadAttention(
              num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
          )(x1, x1)

          # Skip connection
          x2 = layers.Add()([attention_output, temp])

          # Layer Normalization and MLP
          x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
          x3 = keras.Sequential(
              [
                  layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                  layers.Dense(units=embed_dim, activation=tf.nn.gelu),
              ]
          )(x3)

          # Skip connection
          temp = layers.Add()([x3, x2])
      # print(temp.shape)
      # Layer normalization and Global average pooling.
      representation = layers.LayerNormalization(epsilon=layer_norm_eps)(temp)
      representation = layers.GlobalAvgPool1D(keepdims = True)(temp)
      representation_s[i] = representation
      # print(representation.shape)
      
    # Classify outputs.
    # print(len(representation_s))
    # print(representation_s[0].shape)

    for i in range(INPUT_SHAPE[0] // PATCH_SIZE[0]):
      temp_t = representation_s[i]
      # temp_t = tf.expand_dims(temp_t, axis =1)
      for _ in range(transformer_layers):
          # Layer normalization and MHSA
          x1_t = layers.LayerNormalization(epsilon=1e-6)(temp_t)
          attention_output = layers.MultiHeadAttention(
              num_heads=num_heads, key_dim=embed_dim // num_heads, dropout=0.1
          )(x1_t, x1_t)

          # Skip connection
          x2_t = layers.Add()([attention_output, temp_t])

          # Layer Normalization and MLP
          x3_t = layers.LayerNormalization(epsilon=1e-6)(x2_t)
          x3_t = keras.Sequential(
              [
                  layers.Dense(units=embed_dim * 4, activation=tf.nn.gelu),
                  layers.Dense(units=embed_dim, activation=tf.nn.gelu),
              ]
          )(x3_t)

          # Skip connection
          temp_t = layers.Add()([x3_t, x2_t])
      # print(temp.shape)
      # Layer normalization and Global average pooling.
      representation_t = layers.LayerNormalization(epsilon=layer_norm_eps)(temp)
      representation_t = layers.GlobalAvgPool1D()(temp)
      
      outputs = layers.Dense(units=num_classes, activation="softmax")(representation_t)
    
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model


## Train

In [ ]:

def run_experiment():
    # Initialize model
    model = create_vivit_classifier(
        tubelet_embedder=TubeletEmbedding(
            embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
        ),
        positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
    )

    # Compile the model with the optimizer, loss function
    # and the metrics.
    optimizer = keras.optimizers.Adam(learning_rate=LEARNING_RATE)
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

    # Train the model.
    _ = model.fit(trainloader, epochs=EPOCHS, validation_data=validloader)

    _, accuracy, top_5_accuracy = model.evaluate(testloader)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")
    print(f"Test top 5 accuracy: {round(top_5_accuracy * 100, 2)}%")

    return model


# model = create_vivit_classifier(
#         tubelet_embedder=TubeletEmbedding(
#             embed_dim=PROJECTION_DIM, patch_size=PATCH_SIZE
#         ),
#         positional_encoder=PositionalEncoder(embed_dim=PROJECTION_DIM),
#     )
model = run_experiment()

Epoch 1/60
31/31 [==============================] - 19s 32ms/step - loss: 2.3018 - accuracy: 0.1379 - top-5-accuracy: 0.5947 - val_loss: 2.1971 - val_accuracy: 0.2609 - val_top-5-accuracy: 0.6025
Epoch 2/60
31/31 [==============================] - 1s 22ms/step - loss: 2.0839 - accuracy: 0.2366 - top-5-accuracy: 0.7479 - val_loss: 1.8984 - val_accuracy: 0.3602 - val_top-5-accuracy: 0.8137
Epoch 3/60
31/31 [==============================] - 1s 24ms/step - loss: 1.8702 - accuracy: 0.3261 - top-5-accuracy: 0.8405 - val_loss: 1.6013 - val_accuracy: 0.4410 - val_top-5-accuracy: 0.9255
Epoch 4/60
31/31 [==============================] - 1s 24ms/step - loss: 1.6695 - accuracy: 0.4033 - top-5-accuracy: 0.9012 - val_loss: 1.3558 - val_accuracy: 0.5342 - val_top-5-accuracy: 0.9503
Epoch 5/60
31/31 [==============================] - 1s 17ms/step - loss: 1.5170 - accuracy: 0.4331 - top-5-accuracy: 0.9239 - val_loss: 1.1815 - val_accuracy: 0.6335 - val_top-5-accuracy: 0.9627
Epoch 6/60
31/31 [======

## Inference

In [ ]:
NUM_SAMPLES_VIZ = 25
testsamples, labels = next(iter(testloader))
testsamples, labels = testsamples[:NUM_SAMPLES_VIZ], labels[:NUM_SAMPLES_VIZ]

ground_truths = []
preds = []
videos = []

for i, (testsample, label) in enumerate(zip(testsamples, labels)):
    # Generate gif
    with io.BytesIO() as gif:
        imageio.mimsave(gif, (testsample.numpy() * 255).astype("uint8"), "GIF", fps=5)
        videos.append(gif.getvalue())

    # Get model prediction
    output = model.predict(tf.expand_dims(testsample, axis=0))[0]
    pred = np.argmax(output, axis=0)

    ground_truths.append(label.numpy().astype("int"))
    preds.append(pred)


def make_box_for_grid(image_widget, fit):
    """Make a VBox to hold caption/image for demonstrating option_fit values.

    Source: https://ipywidgets.readthedocs.io/en/latest/examples/Widget%20Styling.html
    """
    # Make the caption
    if fit is not None:
        fit_str = "'{}'".format(fit)
    else:
        fit_str = str(fit)

    h = ipywidgets.HTML(value="" + str(fit_str) + "")

    # Make the green box with the image widget inside it
    boxb = ipywidgets.widgets.Box()
    boxb.children = [image_widget]

    # Compose into a vertical box
    vb = ipywidgets.widgets.VBox()
    vb.layout.align_items = "center"
    vb.children = [h, boxb]
    return vb


boxes = []
for i in range(NUM_SAMPLES_VIZ):
    ib = ipywidgets.widgets.Image(value=videos[i], width=100, height=100)
    true_class = info["label"][str(ground_truths[i])]
    pred_class = info["label"][str(preds[i])]
    caption = f"T: {true_class} | P: {pred_class}"

    boxes.append(make_box_for_grid(ib, caption))

ipywidgets.widgets.GridBox(
    boxes, layout=ipywidgets.widgets.Layout(grid_template_columns="repeat(5, 200px)")
)

1/1 [==============================] - 0s 21ms/step


GridBox(children=(VBox(children=(HTML(value="'T: pancreas | P: pancreas'"), Box(children=(Image(value=b'GIF89a…